In [5]:
import json
import requests
import webbrowser
import base64

client_id = ''
client_secret = ''
redirect_url = 'https://www.dialogue.co'
scope = 'offline_access accounting.transactions.read accounting.reports.read accounting.journals.read accounting.settings.read accounting.contacts.read accounting.attachments.read files.read assets.read projects.read'
b64_id_secret = base64.b64encode(bytes(client_id + ':' + client_secret, 'utf-8')).decode('utf-8')

def rotate_refresh_tokens(refresh_token):
    # TODO write this refresh token to Amazon Key Management System
    print("Rotating refresh token on AWS")
    return


# Use this first Auth function to get an access token and refresh token for use by Tap Xero
def XeroFirstAuth():
    # 1. Send a user to authorize your app
    auth_url = ('''https://login.xero.com/identity/connect/authorize?''' +
                '''response_type=code''' +
                '''&client_id=''' + client_id +
                '''&redirect_uri=''' + redirect_url +
                '''&scope=''' + scope +
                '''&state=123''')
    webbrowser.open_new(auth_url)
    
    # 2. Users are redirected back to you with a code
    auth_res_url = input('What is the response URL? ')
    start_number = auth_res_url.find('code=') + len('code=')
    end_number = auth_res_url.find('&scope')
    auth_code = auth_res_url[start_number:end_number]
    print('auth code:')
    print(auth_code)
    print('\n')
    
    # 3. Exchange the code
    exchange_code_url = 'https://identity.xero.com/connect/token'
    response = requests.post(exchange_code_url, 
                            headers = {
                                'Authorization': 'Basic ' + b64_id_secret
                            },
                            data = {
                                'grant_type': 'authorization_code',
                                'code': auth_code,
                                'redirect_uri': redirect_url
                            })
    json_response = response.json()
    
    # 4. Receive your tokens
    access_token = json_response['access_token']
    refresh_token = json_response['refresh_token']
    
    # 5. Write token to Amazon Key Management System 
    rotate_refresh_tokens(refresh_token)
    
    return [json_response['access_token'], json_response['refresh_token']]


In [6]:
XeroFirstAuth()

What is the response URL? https://www.dialogue.co/en?code=a6e27eb44bc457979b9363d5b15088ba14a194030253143bb76b3664a18700d4&scope=accounting.transactions.read%20accounting.reports.read%20accounting.journals.read%20accounting.settings.read%20accounting.contacts.read%20accounting.attachments.read%20files.read%20assets.read%20projects.read&state=123
auth code:
a6e27eb44bc457979b9363d5b15088ba14a194030253143bb76b3664a18700d4


{'access_token': 'eyJhbGciOiJSUzI1NiIsImtpZCI6IjFDQUY4RTY2NzcyRDZEQzAyOEQ2NzI2RkQwMjYxNTgxNTcwRUZDMTkiLCJ0eXAiOiJKV1QiLCJ4NXQiOiJISy1PWm5jdGJjQW8xbkp2MENZVmdWY09fQmsifQ.eyJuYmYiOjE1ODUyMzU5NTEsImV4cCI6MTU4NTIzNzc1MSwiaXNzIjoiaHR0cHM6Ly9pZGVudGl0eS54ZXJvLmNvbSIsImF1ZCI6Imh0dHBzOi8vaWRlbnRpdHkueGVyby5jb20vcmVzb3VyY2VzIiwiY2xpZW50X2lkIjoiNUEyMjcxMTc2RUQxNDc2MkI5RkJGN0Y0RkNCMTE2RDYiLCJzdWIiOiJiMjEzODAxZjFhNTc1YTk3OTEyNGY1NWI5ZWQyNzdkZiIsImF1dGhfdGltZSI6MTU4NTIzNTY3OCwieGVyb191c2VyaWQiOiI5MDVlOWRmYS0zMjY2LTQ3MDAtYjFjMi1lN2RlMDU5ZmVlZTciLCJnbG9iYWxfc2Vzc2lvbl9pZCI6IjVkOWQxN

['eyJhbGciOiJSUzI1NiIsImtpZCI6IjFDQUY4RTY2NzcyRDZEQzAyOEQ2NzI2RkQwMjYxNTgxNTcwRUZDMTkiLCJ0eXAiOiJKV1QiLCJ4NXQiOiJISy1PWm5jdGJjQW8xbkp2MENZVmdWY09fQmsifQ.eyJuYmYiOjE1ODUyMzU5NTEsImV4cCI6MTU4NTIzNzc1MSwiaXNzIjoiaHR0cHM6Ly9pZGVudGl0eS54ZXJvLmNvbSIsImF1ZCI6Imh0dHBzOi8vaWRlbnRpdHkueGVyby5jb20vcmVzb3VyY2VzIiwiY2xpZW50X2lkIjoiNUEyMjcxMTc2RUQxNDc2MkI5RkJGN0Y0RkNCMTE2RDYiLCJzdWIiOiJiMjEzODAxZjFhNTc1YTk3OTEyNGY1NWI5ZWQyNzdkZiIsImF1dGhfdGltZSI6MTU4NTIzNTY3OCwieGVyb191c2VyaWQiOiI5MDVlOWRmYS0zMjY2LTQ3MDAtYjFjMi1lN2RlMDU5ZmVlZTciLCJnbG9iYWxfc2Vzc2lvbl9pZCI6IjVkOWQxNDMzMDdhMDQ0OGE5ZGI2NzExMmFjMzliNzMzIiwianRpIjoiYjFhNjUxNWZlZWU0OWU1NjdjY2Y3ZThmNTMzYTQ4MjUiLCJzY29wZSI6WyJhY2NvdW50aW5nLnJlcG9ydHMucmVhZCIsImFjY291bnRpbmcuYXR0YWNobWVudHMucmVhZCIsInByb2plY3RzLnJlYWQiLCJhY2NvdW50aW5nLnNldHRpbmdzLnJlYWQiLCJmaWxlcy5yZWFkIiwiYWNjb3VudGluZy5qb3VybmFscy5yZWFkIiwiYWNjb3VudGluZy50cmFuc2FjdGlvbnMucmVhZCIsImFzc2V0cy5yZWFkIiwiYWNjb3VudGluZy5jb250YWN0cy5yZWFkIiwib2ZmbGluZV9hY2Nlc3MiXX0.pL9LXNB1gsQ3RbP6lklEKz3BU6IBEtH

In [3]:
# 6. Check the full set of tenants you've been authorized to access
def XeroTenants(access_token):
    connections_url = 'https://api.xero.com/connections'
    response = requests.get(connections_url,
                           headers = {
                               'Authorization': 'Bearer ' + access_token,
                               'Content-Type': 'application/json'
                           })
    json_response = response.json()
    print(json_response)
    
    for tenants in json_response:
        json_dict = tenants
    return json_dict['tenantId']

In [4]:
# 7.1 Refreshing access tokens
def XeroRefreshToken(refresh_token):
    token_refresh_url = 'https://identity.xero.com/connect/token'
    response = requests.post(token_refresh_url,
                            headers = {
                                'Authorization' : 'Basic ' + b64_id_secret,
                                'Content-Type': 'application/x-www-form-urlencoded'
                            },
                            data = {
                                'grant_type' : 'refresh_token',
                                'refresh_token' : refresh_token
                            })
    json_response = response.json()
    print(json_response)
    
    new_refresh_token = json_response['refresh_token']
#     rt_file = open('C:/folder/refresh_token.txt', 'w')
#     rt_file.write(new_refresh_token)
#     rt_file.close()
    
    return [json_response['access_token'], json_response['refresh_token']]

In [0]:
# 7.2 Call the API
def XeroRequests():
    old_refresh_token = open('C:/folder/refresh_token.txt', 'r').read()
    new_tokens = XeroRefreshToken(old_refresh_token)
    xero_tenant_id = XeroTenants(new_tokens[0])
    
    get_url = 'https://api.xero.com/api.xro/2.0/Invoices'
    response = requests.get(get_url,
                           headers = {
                               'Authorization': 'Bearer ' + tokens[0],
                               'Xero-tenant-id': xero_tenant_id,
                               'Accept': 'application/json'
                           })
    json_response = response.json()
    print(json_response)
    
    xero_output = open('C:/folder/xero_output.txt', 'w')
    xero_output.write(response.text)
    xero_output.close()